# Projet numérique III : câble sous-marin

#### Louis Blazejczak, Emmanuel Gardin

## Questions théoriques

#### 1.
Le théorème central limite nous permet d'affirmer la convergence de la variable aléatoire constituée de la moyenne arithmétique des variables simulant les situations.

#### 2.


